In [1]:
import trimesh
from PIL import Image
import torch 
import os 

# pytorch3d pipeline:render normal from ground truth mesh
from lib.common.render import Render, cleanShader
from lib.evaluator.evaluator_util import *

import os 
import numpy as np
from pytorch3d.renderer import (
    FoVOrthographicCameras,
    RasterizationSettings,
    MeshRasterizer,
    BlendParams,
    MeshRenderer,
    look_at_view_transform,
    OrthographicCameras,
    PointsRasterizationSettings,
    PointsRenderer,
    PointsRasterizer,
    AlphaCompositor,
)

os.environ["CUDA_VISIBLE_DEVICES"] = "2"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def get_camera(scale, device):
    R, T = look_at_view_transform(20, 0, 0)

    camera = FoVOrthographicCameras(
        device=device,
        R=R,
        T=T,
        znear=100.0,
        zfar=-100.0,
        max_y=100.0,
        min_y=-100.0,
        max_x=100.0,
        min_x=-100.0,
        scale_xyz=(scale * np.ones(3), ),
    )

    return camera

def init_renderer(cam, device):
    raster_settings_mesh = RasterizationSettings(
                image_size=512,
                blur_radius=np.log(1.0 / 1e-4) * 1e-7,
                faces_per_pixel=30,
            )
    meshRas = MeshRasterizer(cameras=cam, raster_settings=raster_settings_mesh)
    blendparam = BlendParams(1e-4, 1e-4, (0.0, 0.0, 0.0))
    renderer = MeshRenderer(
            rasterizer=meshRas,
            shader=cleanShader(device=device, cameras=cam, blend_params=blendparam),
        )
    return renderer

def project_mesh(render, mesh, calib=None, scale=100.0):
    if calib is not None:
        verts_gt = torch.as_tensor(mesh.vertices * scale).float()
        proj_verts = projection(verts_gt, calib)
        proj_verts[:, 1] *= -1
    else:
        proj_verts = torch.as_tensor(mesh.vertices).float()
    faces_gt = torch.as_tensor(mesh.faces).long()

    proj_mesh = render.VF2Mesh(proj_verts, faces_gt)
    return proj_mesh

def get_normal_img(renderer, mesh):
    rendered_img = (
        renderer(mesh[0])[0:1, :, :, :3] - 0.5
    ) * 2.0
    rendered_img = ((rendered_img + 1.0) * 0.5)[0]
    return rendered_img


In [8]:
# pifuhd metrices calc
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

mesh_dir = "/mnt/local4T/pengfei/projects/PointHuman/PointHuman-ICON/data/cape/scans"
subject_names = sorted([i.split('.')[0] for i in os.listdir(mesh_dir)])
views = [
    '000', '120', '240'
]
icon_calib_dir =  "/mnt/local4T/pengfei/projects/PointHuman/PointHuman-ICON/data/cape_3views/"
calib_dir =  "/mnt/local4T/pengfei/projects/PointHuman/PointHuman-ICON/data/cape/pifuhd_gen_512"
output_dir = "/mnt/local4T/pengfei/projects/PointHuman/PointHuman-ICON/results"
pred_mesh_dir = "/mnt/local4T/pengfei/projects/PointHuman/PointHuman-ICON/results"

# mesh normal rendering
scale = 100.0
camera = get_camera(scale, device)
icon_renderer = init_renderer(camera, device)
render = Render(size=512, device=device)

radius = 0.005
calc_mesh_dist = True
calc_NC = True
output_normal_map = True


from PIL import Image
from tqdm import tqdm
method = "open-pifuhd"

# import trimesh

score = []
for subject_name in tqdm(subject_names):
    mesh_path = os.path.join(mesh_dir, subject_name, f"{subject_name}.obj")
    mesh = trimesh.load(mesh_path)
    for view in views:
        # calib_path = os.path.join(calib_dir, subject_name, "PARAM", subject_name, f'{int(view)}_0_00.npy')
        # calib, scale = get_pifuhd_calib(calib_path)

        calib_path = os.path.join(icon_calib_dir, subject_name, "calib", f'{view}.txt')
        calib = load_calib(calib_path)
        if calc_NC:
            # to ndc 
            gt_mesh = project_mesh(render, mesh, calib, scale)
            gt_normal_imgs = get_normal_img(icon_renderer, gt_mesh)
        
        if calc_mesh_dist:
            gt_pcl, gt_vertices, gt_faces = get_proj_pcls(mesh_path, calib)
            # trimesh.Trimesh(gt_pcl).export('gt_proj.obj')

        tmp_dict = {}
        tmp_dict['subject_name'] = subject_name
        tmp_dict['view'] = view
        pred_mesh_path = os.path.join(pred_mesh_dir, method, f'cape/{subject_name}/est_mesh_{view}.obj')
        if calc_NC:
            pred_mesh = trimesh.load(pred_mesh_path)
            pred_mesh = project_mesh(render, pred_mesh, calib=None)
            pred_normal_imgs = get_normal_img(icon_renderer, pred_mesh)
            normal_img_output = os.path.join(pred_mesh_dir, method, f'normal_map_from_mesh/{view}')
            
            error = (((pred_normal_imgs - gt_normal_imgs)**2).sum(dim=2).mean())
            tmp_dict[f'{method}_nc']=error.cpu().numpy()

            if output_normal_map:
                if not os.path.exists(normal_img_output):
                    os.makedirs(normal_img_output)
                
                Image.fromarray(
                    (
                        torch.cat([pred_normal_imgs, gt_normal_imgs], dim=1).cpu().numpy() * 255.0
                    ).astype(np.uint8)
                ).save(os.path.join(normal_img_output, f'{subject_name}.jpg'))
        if calc_mesh_dist:
            pred_pcl, vertices, faces = get_proj_pcls(pred_mesh_path)
            # trimesh.Trimesh(pred_pcl).export('pred_proj.obj')

            if calc_mesh_dist :
                chamfer_dist = get_chamfer_distance(gt_pcl, pred_pcl).cpu().numpy()
                p2s_dist = get_p2s_distance(gt_vertices, gt_faces, pred_pcl).cpu().numpy()
                tmp_dict[f'{method}_chamfer'] = chamfer_dist
                tmp_dict[f'{method}_p2s'] = p2s_dist
        score.append(tmp_dict)
        # break
    # break
    


  0%|          | 0/150 [00:00<?, ?it/s]

100%|██████████| 150/150 [28:40<00:00, 11.47s/it]


In [10]:
output_root = "/mnt/local4T/pengfei/projects/PointHuman/PointHuman-ICON/results"

npy_path = os.path.join(output_root, f'pifuhd_cape_metrices.npy')
np.save(npy_path, score, allow_pickle=True)

In [13]:
output_root = "/mnt/local4T/pengfei/projects/PointHuman/PointHuman-ICON/results/pifuhd_cape_NC_0.005_mesh_icon_calib.npy"

score1 = np.load(output_root,allow_pickle=True).tolist()

[{'subject_name': '00032-longshort-flying_eagle-000240',
  'view': '000',
  'open-pifuhd_nc': array(0.00848278, dtype=float32),
  'open-pifuhd_chamfer': array(5.4083734, dtype=float32),
  'open-pifuhd_p2s': array(2.5117412, dtype=float32)},
 {'subject_name': '00032-longshort-flying_eagle-000240',
  'view': '120',
  'open-pifuhd_nc': array(0.00915843, dtype=float32),
  'open-pifuhd_chamfer': array(6.1035404, dtype=float32),
  'open-pifuhd_p2s': array(2.3885825, dtype=float32)},
 {'subject_name': '00032-longshort-flying_eagle-000240',
  'view': '240',
  'open-pifuhd_nc': array(0.00804076, dtype=float32),
  'open-pifuhd_chamfer': array(5.998455, dtype=float32),
  'open-pifuhd_p2s': array(2.262276, dtype=float32)},
 {'subject_name': '00032-longshort-hips-000150',
  'view': '000',
  'open-pifuhd_nc': array(0.00757024, dtype=float32),
  'open-pifuhd_chamfer': array(6.2518344, dtype=float32),
  'open-pifuhd_p2s': array(2.8086162, dtype=float32)},
 {'subject_name': '00032-longshort-hips-000150

In [14]:
split_path = "/mnt/local4T/pengfei/projects/PointHuman/PointHuman-ICON/data/cape/test150.txt"

data = []
with open(split_path, 'r', encoding='utf-8') as f:
    for ann in f.readlines():
        ann = ann.strip('\n')       #去除文本中的换行符
        ann = ann.split('/')[1]
        data.append(ann)
simple_subject_name = data[:50]
hard_subject_name = data[50:]

import pandas as pd 
# df = pd.DataFrame(score)
df = pd.DataFrame(score1)

easy_df = df.loc[df['subject_name'].isin(simple_subject_name)]
hard_df = df.loc[df['subject_name'].isin(hard_subject_name)]

print(f"easy objs num:{len(easy_df)}, hard objs num:{len(hard_df)}")

tmp_dict = {

}
calc_mesh_dist = True
if calc_mesh_dist and calc_NC:
    tmp_dict[method] = {
        'nc_easy': easy_df[f'{method}_nc'].mean(),
        'nc_hard': hard_df[f'{method}_nc'].mean(),
        'chamfer_easy': easy_df[f'{method}_chamfer'].mean(),
        'chamfer_hard': hard_df[f'{method}_chamfer'].mean(),
        'p2s_easy': easy_df[f'{method}_p2s'].mean(),
        'p2s_hard': hard_df[f'{method}_p2s'].mean(),
    }
elif calc_NC:
    tmp_dict[method] = {
        'nc_easy': easy_df[f'{method}_nc'].mean(),
        'nc_hard': hard_df[f'{method}_nc'].mean(),
    }
else: 
    tmp_dict[method] = {
        'chamfer_easy': easy_df[f'{method}_chamfer'].mean(),
        'chamfer_hard': hard_df[f'{method}_chamfer'].mean(),
        'p2s_easy': easy_df[f'{method}_p2s'].mean(),
        'p2s_hard': hard_df[f'{method}_p2s'].mean(),
    }
tmp_dict

easy objs num:150, hard objs num:300


{'open-pifuhd': {'nc_easy': 0.0063760924339294435,
  'nc_hard': 0.008668893178304037,
  'chamfer_easy': 5.6213981119791665,
  'chamfer_hard': 8.536461588541666,
  'p2s_easy': 2.267054036458333,
  'p2s_hard': 3.1294905598958334}}

In [7]:
# ours (radius=0.005) others render from mesh
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

mesh_dir = "/mnt/local4T/pengfei/projects/PointHuman/PointHuman-ICON/data/cape/scans"
subject_names = sorted([i.split('.')[0] for i in os.listdir(mesh_dir)])
views = [
    '000', '120', '240'
]
calib_dir =  "/mnt/local4T/pengfei/projects/PointHuman/PointHuman-ICON/data/cape_3views/"
output_dir = "/mnt/local4T/pengfei/projects/PointHuman/PointHuman-ICON/results"
pred_mesh_dir = "/mnt/local4T/pengfei/projects/PointHuman/PointHuman-ICON/results"
pifuhd_calib_dir = "/mnt/local4T/pengfei/projects/PointHuman/PointHuman-ICON/data/cape/pifuhd_gen_512"
compare_methods = [
    'icon-filter',
    'icon-nofilter',
    'pamir',
    'pifu',
    'pointhuman_1_outputs',
    'econ'
]

view2obj = {
    '000': 'est_mesh_0.obj',
    '120': 'est_mesh_120.obj',
    '240': 'est_mesh_240.obj',
}

# mesh normal rendering
scale = 100.0
camera = get_camera(scale, device)
icon_renderer = init_renderer(camera, device)
render = Render(size=512, device=device)

radius = 0.005
calc_mesh_dist = True
calc_NC = True
output_normal_map = True

def init_point_renderer(radius, device):
    R, T = look_at_view_transform(20, 0, 0)
    cameras = OrthographicCameras(device=device, R=R, T=T)
    raster_settings = PointsRasterizationSettings(
        image_size=512, 
        radius = radius,
        points_per_pixel = 10
    )

    point_rasterizer = PointsRasterizer(cameras=cameras, raster_settings=raster_settings)
    point_renderer = PointsRenderer(
        rasterizer=point_rasterizer,
        compositor=AlphaCompositor(background_color=(0, 0, 0))
    )

    return point_renderer

# point cloud renderer 
pcls_renderer = init_point_renderer(radius, device)

score = []
from tqdm import tqdm
for subject_name in tqdm(subject_names):
    mesh_path = os.path.join(mesh_dir, subject_name, f"{subject_name}.obj")
    mesh = trimesh.load(mesh_path)
    for view in views:
        calib_path = os.path.join(calib_dir, subject_name, "calib", f'{view}.txt')
        calib = load_calib(calib_path)
        if calc_NC:
            # to ndc 
            gt_mesh = project_mesh(render, mesh, calib, scale)
            gt_normal_imgs = get_normal_img(icon_renderer, gt_mesh)

        # for chamfer and p2s
        if calc_mesh_dist:
            gt_pcl, gt_vertices, gt_faces = get_proj_pcls(mesh_path, calib)

        normal_imgs = [gt_normal_imgs]
        tmp_dict = {}
        tmp_dict['subject_name'] = subject_name
        tmp_dict['view'] = view
        for method in compare_methods:
            if 'pointhuman' not in method:
                # render normal from pointclouds

                # pred_mesh_path = os.path.join(pred_mesh_dir, method, f'cape/{subject_name}/{view2obj[view]}')
                # pred_pcl, _, _ = get_proj_pcls(pred_mesh_path)
                # pred_normal_imgs = get_pcls_normal_map(pcls_renderer, pred_pcl)

                # render normal from mesh 
                pred_mesh_path = os.path.join(pred_mesh_dir, method, f'cape/{subject_name}/{view2obj[view]}')
                if 'econ' in method:
                    pred_mesh_path = os.path.join(pred_mesh_dir, method, f'cape/{subject_name}/cape-{subject_name}-{int(view):03d}_final.obj')
                
                if calc_NC:
                    pred_mesh = trimesh.load(pred_mesh_path)
                    pred_mesh = project_mesh(render, pred_mesh, calib=None)
                    pred_normal_imgs = get_normal_img(icon_renderer, pred_mesh)
                    normal_img_output = os.path.join(pred_mesh_dir, method, f'normal_map_from_mesh/{view}')
                
                if calc_mesh_dist:
                    pred_pcl, _, _ = get_proj_pcls(pred_mesh_path)
                    trimesh.Trimesh(gt_pcl).export(f'{method}_pred_proj.obj')
            else:
                pred_mesh_path = os.path.join(pred_mesh_dir, method, f'cape/{subject_name}/{view}/est_scan.obj')
                
                pred_pcl, _, _ = get_ori_pcls(pred_mesh_path, calib_path, z_norm=False)
                pred_normal_imgs = get_pcls_normal_map(pcls_renderer, pred_pcl)
                normal_img_output = os.path.join(pred_mesh_dir, method, f'normal_map_{radius}/{view}')
            if calc_NC:
                error = (((pred_normal_imgs - gt_normal_imgs)**2).sum(dim=2).mean())
                tmp_dict[f'{method}_nc']=error.cpu().numpy()
            
            if output_normal_map:
                if not os.path.exists(normal_img_output):
                    os.makedirs(normal_img_output)
                Image.fromarray(
                    (
                        pred_normal_imgs.cpu().numpy() * 255.0
                    ).astype(np.uint8)
                ).save(os.path.join(normal_img_output, f'{subject_name}.jpg'))
            if calc_mesh_dist :
                chamfer_dist = get_chamfer_distance(gt_pcl, pred_pcl).cpu().numpy()
                p2s_dist = get_p2s_distance(gt_vertices, gt_faces, pred_pcl).cpu().numpy()
                tmp_dict[f'{method}_chamfer'] = chamfer_dist
                tmp_dict[f'{method}_p2s'] = p2s_dist
        score.append(tmp_dict)
        break
    break
    
# output_root = "/mnt/local4T/pengfei/projects/PointHuman/PointHuman-ICON/results"

# npy_path = os.path.join(output_root, f'econ_cape_NC_{radius}_mesh.npy')
# np.save(npy_path, score, allow_pickle=True)

  0%|          | 0/150 [00:00<?, ?it/s]

  0%|          | 0/150 [00:06<?, ?it/s]


ValueError: string is not a file: /mnt/local4T/pengfei/projects/PointHuman/PointHuman-ICON/results/pointhuman_1_outputsecon/cape/00032-longshort-flying_eagle-000240/000/est_scan.obj

In [26]:
split_path = "/mnt/local4T/pengfei/projects/PointHuman/PointHuman-ICON/data/cape/test150.txt"

data = []
with open(split_path, 'r', encoding='utf-8') as f:
    for ann in f.readlines():
        ann = ann.strip('\n')       #去除文本中的换行符
        ann = ann.split('/')[1]
        data.append(ann)
simple_subject_name = data[:50]
hard_subject_name = data[50:]

import pandas as pd 
df = pd.DataFrame(score)

easy_df = df.loc[df['subject_name'].isin(simple_subject_name)]
hard_df = df.loc[df['subject_name'].isin(hard_subject_name)]

print(f"easy objs num:{len(easy_df)}, hard objs num:{len(hard_df)}")

tmp_dict = {

}
calc_mesh_dist = True
for method in compare_methods:
    if calc_mesh_dist and calc_NC:
        tmp_dict[method] = {
            'nc_easy': easy_df[f'{method}_nc'].mean(),
            'nc_hard': hard_df[f'{method}_nc'].mean(),
            'chamfer_easy': easy_df[f'{method}_chamfer'].mean(),
            'chamfer_hard': hard_df[f'{method}_chamfer'].mean(),
            'p2s_easy': easy_df[f'{method}_p2s'].mean(),
            'p2s_hard': hard_df[f'{method}_p2s'].mean(),
        }
    elif calc_NC:
        tmp_dict[method] = {
            'nc_easy': easy_df[f'{method}_nc'].mean(),
            'nc_hard': hard_df[f'{method}_nc'].mean(),
        }
    else: 
        tmp_dict[method] = {
            'chamfer_easy': easy_df[f'{method}_chamfer'].mean(),
            'chamfer_hard': hard_df[f'{method}_chamfer'].mean(),
            'p2s_easy': easy_df[f'{method}_p2s'].mean(),
            'p2s_hard': hard_df[f'{method}_p2s'].mean(),
        }
tmp_dict

easy objs num:150, hard objs num:300


{'econ': {'nc_easy': 0.01136670192082723,
  'nc_hard': 0.011482807795206705,
  'chamfer_easy': 1.991116943359375,
  'chamfer_hard': 2.0399420166015627,
  'p2s_easy': 0.9865234375,
  'p2s_hard': 0.989132080078125}}

In [19]:
import os 
import numpy as np
compare_methods = [
    'icon-filter',
    'icon-nofilter',
    'pamir',
    'pifu',
    # 'pointhuman_1_outputs'
]

output_root = "/mnt/local4T/pengfei/projects/PointHuman/PointHuman-ICON/results"
radius=0.005
npy_path = os.path.join(output_root, f'cape_NC_no_pretrain_{radius}_mesh.npy')
# np.save(npy_path, score, allow_pickle=True)
score = list(np.load(npy_path, allow_pickle=True))


In [22]:
split_path = "/mnt/local4T/pengfei/projects/PointHuman/PointHuman-ICON/data/cape/test150.txt"

data = []
with open(split_path, 'r', encoding='utf-8') as f:
    for ann in f.readlines():
        ann = ann.strip('\n')       #去除文本中的换行符
        ann = ann.split('/')[1]
        data.append(ann)
simple_subject_name = data[:50]
hard_subject_name = data[50:]

import pandas as pd 
df = pd.DataFrame(score)
easy_df = df.loc[df['subject_name'].isin(simple_subject_name)]
hard_df = df.loc[df['subject_name'].isin(hard_subject_name)]

print(f"easy objs num:{len(easy_df)}, hard objs num:{len(hard_df)}")

tmp_dict = {

}
calc_mesh_dist = True
for method in compare_methods:
    if calc_mesh_dist:
        tmp_dict[method] = {
            'nc_easy': easy_df[f'{method}_nc'].mean(),
            'nc_hard': hard_df[f'{method}_nc'].mean(),
            'chamfer_easy': easy_df[f'{method}_chamfer'].mean(),
            'chamfer_hard': hard_df[f'{method}_chamfer'].mean(),
            'p2s_easy': easy_df[f'{method}_p2s'].mean(),
            'p2s_hard': hard_df[f'{method}_p2s'].mean(),
        }
    else: 
        tmp_dict[method] = {
            'chamfer_easy': easy_df[f'{method}_chamfer'].mean(),
            'chamfer_hard': hard_df[f'{method}_chamfer'].mean(),
            'p2s_easy': easy_df[f'{method}_p2s'].mean(),
            'p2s_hard': hard_df[f'{method}_p2s'].mean(),
        }

tmp_dict

easy objs num:150, hard objs num:300


{'icon-filter': {'nc_easy': 0.007810561656951904,
  'nc_hard': 0.008441685040791829,
  'chamfer_easy': 1.9694034830729166,
  'chamfer_hard': 2.396436564127604,
  'p2s_easy': 1.0064949544270834,
  'p2s_hard': 1.2396232096354167},
 'icon-nofilter': {'nc_easy': 0.010004751682281494,
  'nc_hard': 0.010963342984517415,
  'chamfer_easy': 2.3179927571614583,
  'chamfer_hard': 2.6579024251302084,
  'p2s_easy': 1.2059713745117187,
  'p2s_hard': 1.3863036092122396},
 'pamir': {'nc_easy': 0.01780653953552246,
  'nc_hard': 0.019029704729715984,
  'chamfer_easy': 3.695631917317708,
  'chamfer_hard': 3.4548494466145834,
  'p2s_easy': 1.9087520345052083,
  'p2s_hard': 1.7454380289713542},
 'pifu': {'nc_easy': 0.007188405195871989,
  'nc_hard': 0.008046290079752605,
  'chamfer_easy': 3.8220096842447915,
  'chamfer_hard': 4.914906005859375,
  'p2s_easy': 1.9558575439453125,
  'p2s_hard': 2.300049031575521}}

In [ ]:
# test gt mesh projection 

mesh_path = "/mnt/local4T/pengfei/projects/PointHuman/PointHuman-ICON/data/cape/scans/00032-longshort-flying_eagle-000240/00032-longshort-flying_eagle-000240.obj"
calib_path = ""
